# Analyse et Visualisation des Données des Cartes Graphiques

Ce notebook a pour objectif d'analyser et de visualiser les données relatives aux cartes graphiques collectées sur plusieurs plateformes (eBay, Flipkart, Ubuy). 

Les étapes réalisées incluent :

- **Importation des bibliothèques** : Manipulation des données, visualisation et gestion des chemins.
- **Configuration des chemins** : Définition des répertoires du projet pour les données nettoyées et les résultats.
- **Chargement des données nettoyées** : Lecture et préparation des fichiers CSV.
- **Filtrage des produits** : Identification des produits disponibles sur plusieurs plateformes.
- **Analyse et visualisation des tendances de prix** : Agrégation des données et génération de graphiques.

Chaque fonction utilisée est expliquée en détail dans la suite du notebook.


# Importation des Librairies

Dans cette section, nous importons les bibliothèques nécessaires pour :

- La manipulation de données avec **pandas**.
- La création de graphiques avec **matplotlib** et **seaborn**.
- La gestion des chemins avec **os** et **pathlib**.
- Le traitement des expressions régulières avec **re**.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import re

# Configuration des Chemins du Projet

Nous définissons ici les chemins d’accès aux différents répertoires du projet :
- `PROJECT_ROOT` : Racine du projet.
- `CLEANED_DATA_PATH` : Dossier contenant les données nettoyées.
- `RESULTS_PATH` : Dossier où seront sauvegardés les résultats (les graphiques générés).


In [ ]:
# Configure paths
PROJECT_ROOT = Path(__file__).resolve().parents[2]
CLEANED_DATA_PATH = PROJECT_ROOT / 'data' / 'cleaned'
RESULTS_PATH = PROJECT_ROOT / 'results'
RESULTS_PATH.mkdir(exist_ok=True)

# Chargement des Données Nettoyées

La fonction `load_cleaned_data` charge l’ensemble des fichiers CSV nettoyés pour la catégorie **graphics_cards** provenant des plateformes *eBay*, *Flipkart* et *Ubuy*. Pour chaque fichier, elle réalise les opérations suivantes :

- **Standardisation des noms de colonnes** : Conversion en minuscules et remplacement des espaces.
- **Ajout d’une colonne `platform`** : Identification de la plateforme d’origine.
- **Normalisation du modèle GPU** : Application de la fonction `normalize_gpu_model`.
- **Gestion de la date de collecte** : Conversion de la colonne `collection_date` en objet date.
  
Les différents DataFrames sont ensuite concaténés pour constituer un DataFrame global.


In [ ]:
def load_cleaned_data():
    """Load cleaned data for graphics cards from all platforms"""
    platforms = ['ebay', 'flipkart', 'ubuy']
    dfs = []
    
    for platform in platforms:
        path = CLEANED_DATA_PATH / platform / 'graphics_cards' / f'*.csv'
        for file in path.parent.glob(path.name):
            df = pd.read_csv(file)
            
            # Standardize column names
            df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
            
            # Add platform identifier
            df['platform'] = platform
            
            # Normalize Chipset/GPU Model
            df['chipset/gpu_model'] = df['chipset/gpu_model'].apply(normalize_gpu_model)
            
            # Ensure 'collection_date' exists
            if 'collection_date' not in df.columns:
                print(f"Warning: 'collection_date' column missing in {file}. Skipping file.")
                continue
            
            # Convert 'collection_date' to datetime and handle errors
            df['collection_date'] = pd.to_datetime(df['collection_date'], errors='coerce')
            
            # Normalize collection_date to only include the date (ignore time)
            df['collection_date'] = df['collection_date'].dt.date
            
            dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

# Filtrage des Produits Selon leur Disponibilité sur Plusieurs Plateformes

La fonction `filter_products_by_platforms` regroupe les produits en fonction de spécifications clés (*memory_size*, *memory_type*, *chipset/gpu_model*) afin de déterminer sur quelles plateformes chaque produit est disponible.

Elle crée :
- Une colonne `platforms` contenant la liste (triée) des plateformes pour chaque produit.
- Une colonne `platform_count` indiquant le nombre de plateformes disponibles.

Elle renvoie ensuite deux DataFrames :
- `all_platforms` : Produits présents sur les trois plateformes.
- `two_platforms` : Produits présents sur exactement deux plateformes.


In [ ]:
def filter_products_by_platforms(df: pd.DataFrame):
    """Filter products based on their availability across platforms"""
    # Group by key specifications
    grouped = df.groupby(
        ['memory_size', 'memory_type', 'chipset/gpu_model']
    )['platform'].unique().reset_index()
    
    # Add a column indicating the platforms each product is available on
    grouped['platforms'] = grouped['platform'].apply(lambda x: sorted(x))
    grouped['platform_count'] = grouped['platforms'].apply(len)
    
    # Filter products available on all three platforms
    all_platforms = grouped[grouped['platform_count'] == 3]
    
    # Filter products available on any two platforms
    two_platforms = grouped[grouped['platform_count'] == 2]
    
    return all_platforms, two_platforms


# Analyse des Différences de Prix et Visualisation

La fonction `analyze_price_differences` a pour objectif d'analyser les écarts de prix pour les produits disponibles sur plusieurs plateformes et de visualiser l’évolution de ces prix dans le temps.

Les étapes clés sont :

- **Fusion des données** : Association des produits filtrés avec les données originales pour récupérer les colonnes `collection_date` et `price`.
- **Agrégation** : Calcul du prix moyen par jour et par plateforme.
- **Visualisation** : Création de graphiques en barres pour chaque produit, affichant l’évolution des prix au fil des dates de collecte.
- **Sauvegarde** : Enregistrement de chaque graphique dans le dossier des résultats, avec un nom de fichier dynamique dérivé des spécifications du produit.


In [ ]:
def analyze_price_differences(all_platforms_df: pd.DataFrame, two_platforms_df: pd.DataFrame, original_df: pd.DataFrame):
    """Analyze price differences and plot trends for products available on multiple platforms"""
    category_results = RESULTS_PATH / 'graphics_cards'
    category_results.mkdir(exist_ok=True)
    
    # Combine both datasets for analysis
    combined_df = pd.concat([all_platforms_df, two_platforms_df], ignore_index=True)
    
    # If no products are available on multiple platforms, skip analysis
    if combined_df.empty:
        print("No products available on multiple platforms. Skipping analysis.")
        return
    
    # Merge with the original dataframe to retain collection_date and price
    merged_df = pd.merge(
        original_df,
        combined_df[['memory_size', 'memory_type', 'chipset/gpu_model']],
        on=['memory_size', 'memory_type', 'chipset/gpu_model'],
        how='inner'
    )
    
    # Ensure 'collection_date' exists
    if 'collection_date' not in merged_df.columns:
        print("Error: 'collection_date' column missing after merging. Skipping analysis.")
        return
    
    # Drop rows with missing collection_date
    merged_df = merged_df.dropna(subset=['collection_date'])
    
    # Aggregate data by day and platform
    df_aggregated = merged_df.groupby(
        ['memory_size', 'memory_type', 'chipset/gpu_model', 'platform', 'collection_date']
    )['price'].mean().reset_index()
    
    # Sort by date to ensure correct plotting order
    df_aggregated = df_aggregated.sort_values('collection_date')
    
    # Group data for analysis
    grouped = df_aggregated.groupby(
        ['memory_size', 'memory_type', 'chipset/gpu_model', 'platform', 'collection_date']
    )['price'].mean().unstack(level='platform')
    
    # Initialize a counter for dynamic file naming
    product_counter = 1
    
    # Plot column graphs for each product
    for product, data in grouped.groupby(level=[0, 1, 2]):
        # Format Memory Size to remove decimals
        memory_size = str(int(product[0])) if isinstance(product[0], (float, int)) else product[0]
        
        # Construct a default title using product specifications
        product_title = f"{memory_size}_{product[1]}_{product[2]}"
        
        plt.figure(figsize=(12, 6))
        
        # Plot each platform's prices as columns
        data.plot(kind='bar', figsize=(12, 6))
        
        plt.title(f'Price Trends for {product_title}')
        plt.xlabel('Collection Date')
        plt.ylabel('Price (USD)')
        
        # Set x-ticks to only show dates
        plt.xticks(range(len(data.index)), data.index.get_level_values('collection_date'), rotation=45, ha='right')
        
        plt.legend(title='Platform')
        plt.grid(True)
        plt.tight_layout()
        
        # Sanitize the title for file naming
        sanitized_title = (
            "_".join(product_title.split())
            .replace("/", "_")
            .replace("\\", "_")
            .replace(":", "_")
            .replace("*", "_")
            .replace("?", "_")
            .replace('"', "_")
            .replace("<", "_")
            .replace(">", "_")
            .replace("|", "_")
        )
        
        # Save the plot with dynamic file naming
        file_name = f"Product{product_counter:02d}_{sanitized_title}.png"
        plt.savefig(category_results / file_name)
        plt.close()
        
        # Increment the counter for the next product
        product_counter += 1


# Exécution Principale

La dernière section orchestre l’exécution complète de l’analyse :

1. **Chargement des données nettoyées** via `load_cleaned_data()`.
2. **Filtrage des produits** disponibles sur plusieurs plateformes grâce à `filter_products_by_platforms()`.
3. **Analyse des différences de prix** et génération des graphiques par la fonction `analyze_price_differences()`.

En cas d'erreur durant le processus, un message approprié sera affiché.


In [ ]:
if __name__ == "__main__":
    try:
        print("Analyzing graphics cards...")
        
        # Load cleaned data for graphics cards
        df = load_cleaned_data()
        
        # Par exemple, vous pouvez calculer la moyenne des prix pour les doublons ici
        
        # Filter products based on their availability across platforms
        all_platforms_df, two_platforms_df = filter_products_by_platforms(df)
        
        # Affichage d'un résumé sur le nombre de produits identifiés
        print(f"Products available on all platforms: {len(all_platforms_df)}")
        print(f"Products available on any two platforms: {len(two_platforms_df)}")
        
        # Analyze price differences and generate plots
        analyze_price_differences(all_platforms_df, two_platforms_df, df)
    except Exception as e:
        print(f"Error processing graphics cards: {str(e)}")
